In [1]:
import os, errno, sys
from bs4 import BeautifulSoup
from requests import get
from textblob import TextBlob
import pandas as pd
from pyteaser import Summarize
from django.utils.encoding import smart_str, smart_unicode
from scraper_functions import *

proj_path = "/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/site/src/"
# This is so Django knows where to find stuff.
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "manatwee.settings")
sys.path.append(proj_path)

# This is so my local_settings.py gets loaded.
os.chdir(proj_path)

# This is so models get loaded.
from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

from twttr.models import *
from critic.models import *


In [2]:
SLATE_URL = 'http://www.slate.com/articles/arts/movies.html'



In [4]:
SlateSoup = BeautifulSoup(get(SLATE_URL).text,'html.parser')

In [75]:
slate_dict = {}
for item in SlateSoup.find_all('div',class_="tile long-hed stacked"):
    if 'reviewed' in item.find_all('a')[0]['href']:
        try:
            r_url= item.find_all('a')[0]['href']
            urllist.append(r_url)

            r_soup = BeautifulSoup(get(r_url).text,'html.parser')
            r_title = r_soup.find_all('h1',class_='article__hed')[0].findAll('em')[0].text

            print r_url
            print r_title
            
            r_title_slug = r_title.lower().replace(' ','-').replace('(','').replace(')','')
            print r_title_slug
            print r_title_slug in r_url
            if r_title_slug in r_url:
                print 'got to if statement'
                r_dict = {}
                r_text = ''
                for item in r_soup.find_all('p',class_='slate-paragraph'):
                    r_text += item.text
                r_sentiment = TextBlob(r_text).sentiment
                r_polarity = r_sentiment[0]
                r_subjectivity = r_sentiment[1]
                summary = Summarize(r_title,r_text)
                r_blurb = ' '.join(summary)
                r_thumbsup = 'Positive' if r_polarity >0 else 'Negative'
                r_critic = r_soup.find_all('meta',attrs={'name':'author'})[0]['content']
                r_dict['critic'] = r_critic
                r_dict['text'] = r_text
                r_dict['blurb'] = r_blurb
                r_dict['review_subjectivity'] = r_subjectivity
                r_dict['review_polarity'] = r_polarity
                r_dict['sentiment'] = r_sentiment
                r_dict['thumbsup'] = r_thumbsup
                slate_dict[r_title] = r_dict 
                
            
        except:
            print 'movie not found. skipping...'
        

        


https://slate.com/arts/2017/12/the-greatest-showman-the-movie-musical-reviewed.html
The Greatest Showman 
the-greatest-showman-
True
got to if statement
https://slate.com/arts/2017/12/darkest-hour-starring-gary-oldman-reviewed.html
Darkest Hour
darkest-hour
True
got to if statement
https://slate.com/arts/2017/12/call-me-by-your-name-with-armie-hammer-reviewed.html
Call Me by Your Name 
call-me-by-your-name-
True
got to if statement
https://slate.com/arts/2017/12/murder-on-the-orient-express-reviewed.html
Murder on the Orient Express 
murder-on-the-orient-express-
True
got to if statement
https://slate.com/arts/2017/12/thor-ragnarok-the-latest-marvel-movie-reviewed.html
Thor: Ragnarok 
thor:-ragnarok-
False
https://slate.com/arts/2017/12/darren-aronofsky-and-jennifer-lawrences-mother-reviewed.html
Mother!
mother!
False
https://slate.com/arts/2017/12/blade-runner-2049-starring-ryan-gosling-reviewed.html
Blade Runner 2049 
blade-runner-2049-
True
got to if statement
https://slate.com/arts

In [28]:
r_soup = BeautifulSoup(get(urllist[0]).text,'html.parser')

In [45]:
r_soup.find_all('h1',class_='article__hed')[0].findAll('em')[0].text

u'The Greatest Showman '

In [47]:
slate_dict

{u'Goodfellas': {'link': u'https://slate.com/arts/2017/12/i-tonya-the-tonya-harding-movie-reviewed.html',
  'name': u'Goodfellas'}}

In [60]:
r_text = ''
for item in r_soup.find_all('p',class_='slate-paragraph'):
    r_text += item.text

In [63]:
r_soup.find_all('meta',attrs={'name':'author'})[0]['content']

u'Sam Adams'

In [20]:
for item in Reviews.objects.filter(organization=slateorg):
    print item.movie

Call Me by Your Name 
Blade Runner 2049 
Darkest Hour
Lady Bird 
The Greatest Showman 
The Meyerowitz Stories (New and Selected)
Wonderstruck 
The Shape of Water 
Justice League 
Coco 
Phantom Thread 
The Florida Project
American Made 
Mudbound 
The Killing of a Sacred Deer 
The Last Jedi 
Murder on the Orient Express 
Last Flag Flying 


In [6]:
slateorg = Organization.objects.get(name='Slate')

In [23]:
callme=Movie.objects.get(name='Call Me by Your Name')

In [24]:
chi = Organization.objects.get(name='ChicagoTribune')

In [31]:
for mov in Reviews.objects.filter(organization=chi):
    print mov.movie

Thirst Street
BPM
For Ahkeem
Sylvio
Mollys Game
All the Money in the World
Gilbert
Ferdinand
Downsizing
I, Tonya
Darkest Hour
Roman J. Israel, Esq.
Bill Nye: Science Guy
Star Wars: The Last Jedi
The Future Perfect
Una
Jumanji: Welcome to the Jungle
Coco
Call Me by Your Name
Wonder Wheel
The Disaster Artist
Pitch Perfect 3
Father Figures
The Shape of Water
The Ballad of Lefty Brown
The Post
Hostiles


In [35]:
Reviews.objects.filter(movie=callme)

<QuerySet [<Reviews: Set in the summer of 1983, in a land of leisurely alfresco lunches and spontaneous all-day bike rides under the northern Italian sun, the romantic idyll “Call Me by Your Name” is enough to make you  move to the town of Crema, even if your rational self realizes the director Luca Guadagnino trades in a heightened, miragelike state of mythic yearning.The swoony atmosphere is familiar from his earlier films, particularly “I Am Love” (2009), in which Tilda Swinton communed with the Italian countryside, a young lover and every sensory pleasure known to Swintonhood. For the record, Crema’s patron saint is Pantaleon — the patron saint of physicians. The love doctor’s always on call in this corner of the world, and there’s no cure for what ails the young lovers except time, experience, regret and gratitude.In other words: “Call Me by Your Name” is about being 17. Timothee Chalamet, currently on screens as the blithely self-absorbed charmer advocating for “un-special sex” i

In [48]:
slate_callme = Reviews.objects.filter(organization=slateorg)[0]

In [51]:
slate_callme_movie = slate_callme.movie

In [55]:
slate_callme_movie.slug = 'call-me-by-your-name'

In [56]:
slate_callme.movie = slate_callme_movie

In [57]:
slate_callme_movie.save()

saving...
saved


In [58]:
slate_callme.save()

In [46]:
Reviews.objects.filter(organization=chi)[18].movie.slug

u'call-me-by-your-name'

In [41]:
Reviews.objects.filter(organization=slateorg)[0].movie==Reviews.objects.filter(organization=chi)[18].movie

False

In [47]:
for rev in Reviews.objects.filter(organization=slateorg):
    print rev.movie.slug

call-me-by-your-name-myyh
blade-runner-2049
darkest-hour
lady-bird
the-greatest-showman-26au
the-meyerowitz-stories-new-and-selected
wonderstruck
the-shape-of-water-kb9m
justice-league-t0rc
coco-vylp
phantom-thread-xnq3
the-florida-project
american-made
mudbound
the-killing-of-a-sacred-deer
the-last-jedi
murder-on-the-orient-express
last-flag-flying


In [61]:
Reviews.objects.filter(movie=callme)[0].movie.slug

u'call-me-by-your-name'

In [83]:
ladybird = Show.objects.all()[30].slug

In [88]:
ladybird

u'ladybird'

In [89]:
ladybird = Show.objects.all()[31]

In [92]:
ladybird.name='lady bird'

In [95]:
ladybird.slug = 'lady-bird'

In [97]:
ladybird.save()

saving...
2017-12-18 03:32:06.130356+00:00
saved
2017-12-18 03:32:06.130356+00:00
